<a href="https://colab.research.google.com/github/Stacy067/Deep-learning-and-Tensorflow/blob/main/TF_Certificate_Category_3_cats_vs_dogs_transfer_learning_(%EA%B0%95%EC%9D%98)%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Category 3 - Cats vs Dogs 분류

* Convolution Neural network 활용한 분류 모델 (Classification)
* tensorflow-datasets 를 활용한 데이터 전처리

## 확인

1. GPU 옵션 켜져 있는지 확인할 것!!! (수정 - 노트설정 - 하드웨어설정 (GPU))

## 순서

1. **import**: 필요한 모듈 import
2. **전처리**: 학습에 필요한 데이터 전처리를 수행합니다.
3. **모델링(model)**: 모델을 정의합니다.
4. **컴파일(compile)**: 모델을 생성합니다.
5. **학습 (fit)**: 모델을 학습시킵니다.

## 문제

Computer Vision with CNNs
<br>
<br>For this exercise you will build a cats v dogs classifier
<br>using the Cats v Dogs dataset from TFDS.
<br>Be sure to use the final layer as shown 
<br>    **(Dense, 2 neurons, softmax activation)**
<br>
<br>The testing infrastructre will **resize all images to 224x224**
<br>with **3 bytes of color depth**. Make sure your input layer trains
<br>images to that specification, or the tests will fail.
<br>
<br>Make sure your output layer is exactly as specified here, or the 
<br>tests will fail.

----------------------------------------
<br>이 연습에서는 cats v dogs 분류기를 만들 것입니다.
TFDS의 Cats v Dogs 데이터 세트 사용.
<br> 그림과 같이 최종 레이어를 사용하십시오
<br> **(Dense, 뉴런 2 개, activation='softmax')**
<br>
<br> 테스트 인프라는 **모든 이미지의 크기를 224x224로 조정합니다(컬러사진)**. 입력 레이어를 확인하십시오

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16

## Load Dataset

**tensorflow-datasets**를 활용합니다.

* [Cats vs Dogs 데이터셋 문서 보기](https://www.tensorflow.org/datasets/catalog/cats_vs_dogs?hl=ko)

* [tensorflow-datasets 다루기](https://www.tensorflow.org/datasets/splits?hl=ko)

**시험에서 주어지는 데이터셋 로드 형태**

* 예전 방식이므로 아래처러 주어지는 코드를 과감히 삭제 후, 아래 제공되는 방식으로 변경합니다.

In [2]:
setattr(tfds.image_classification.cats_vs_dogs, '_URL',"https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip")

In [3]:
dataset_name = 'cats_vs_dogs'

# 처음 80%의 데이터만 사용
train_dataset = tfds.load(name=dataset_name, split='train[:80%]')

# 최근 20%의 데이터만 사용
valid_dataset = tfds.load(name=dataset_name, split='train[80%:]')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteWJMY9V/cats_vs_dogs-train.tfrecord*...:   0%|    …

Dataset cats_vs_dogs downloaded and prepared to ~/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


시험에서 요구하는 **전처리 요구 조건**은 다음과 같습니다.

1. 이미지 정규화 (Normalization)
2. 이미지 사이즈 맞추기: (224 X 224) 
3. image(x), label(y)를 분할

**[실습코드]**

In [4]:
def preprocess(data):
    # x, y 데이터를 정의합니다.
    x = data['image']
    y = data['label']
    # image 정규화(Normalization)
    x = x / 255
    # 사이즈를 (224, 224)로 변환합니다.
    x = tf.image.resize(x, size=(224, 224))
    # x, y  데이터를 return 합니다.
    return x, y

만든 전처리 함수(preprocessing)를 **dataset에 mapping**하고, **batch_size도 지정**합니다.

In [5]:
batch_size=32

In [6]:
train_data = train_dataset.map(preprocess).batch(batch_size)
valid_data = valid_dataset.map(preprocess).batch(batch_size)

## 모델 정의 (Sequential) 
: 여기부분이 달라짐

이제 Modeling을 할 차례입니다.

`Sequential` 모델 안에서 층을 깊게 쌓아 올려 주면 됩니다.

1. `input_shape`는 224 X 224 컬러사진인 **(224, 224, 3)**으로 지정합니다.
2. transfer learning 기법을 통해 VGG16 모델을 활용한 전이학습 모델을 완성합니다.
3. 출력층은 class 갯수 2개의 뉴런이 요구됩니다.


In [7]:
transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) 
#imagenet의 학습 분류기를 가져와라, include_top VGG 테이블 아래부분 안가져오겠다, 여기 부분에서 전이학습
transfer_model.trainable=False # 가중치를 freeze

58900480/58889256 [==============================] - 0s 0us/step


In [8]:
model = Sequential([
    transfer_model, #전이학습 들어갈때
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax'),
])

In [9]:
model.summary() # Trainable params는 학습해야하는 모델 개수,  Non-trainable params는 vgg16

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dropout (Dropout)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 512)               12845568  
                                                                 
 dense_1 (Dense)             (None, 128)               65664     
                                                                 
 dense_2 (Dense)             (None, 2)                 258       
                                                                 
Total params: 27,626,178
Trainable params: 12,911,490
No

## 컴파일 (compile)

1. `optimizer`는 가장 최적화가 잘되는 알고리즘인 'adam'을 사용합니다.
2. `loss`설정
  * 출력층 activation이 `sigmoid` 인 경우: `binary_crossentropy`
  * 출력층 activation이 `softmax` 인 경우: 
    * 원핫인코딩(O): `categorical_crossentropy`
    * 원핫인코딩(X): `sparse_categorical_crossentropy`)
3. `metrics`를 'acc' 혹은 'accuracy'로 지정하면, 학습시 정확도를 모니터링 할 수 있습니다.

전처리 단계에서 **one-hot encoding** 을 해주었습니다. 따라서, `categorical_crossentropy`를 지정해주면 됩니다.

model.compile()

In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

## ModelCheckpoint: 체크포인트 생성

`val_loss` 기준으로 epoch 마다 최적의 모델을 저장하기 위하여, ModelCheckpoint를 만듭니다.
* `checkpoint_path`는 모델이 저장될 파일 명을 설정합니다.
* `ModelCheckpoint`을 선언하고, 적절한 옵션 값을 지정합니다.

In [11]:
checkpoint_path = "my_checkpoint.ckpt"
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                             save_weights_only=True, 
                             save_best_only=True, 
                             monitor='val_loss', 
                             verbose=1)

## 학습 (fit)

1. `validation_data`를 반드시 지정합니다.
2. `epochs`을 적절하게 지정합니다.
3. `callbacks`에 바로 위에서 만든 checkpoint를 지정합니다.

In [12]:
model.fit(train_data,
          validation_data=(valid_data),
          epochs=20,
          callbacks=[checkpoint],
          )

Epoch 1/20
582/582 [==============================] - ETA: 0s - loss: 0.2918 - acc: 0.8836
Epoch 1: val_loss improved from inf to 0.17412, saving model to my_checkpoint.ckpt
582/582 [==============================] - 138s 213ms/step - loss: 0.2918 - acc: 0.8836 - val_loss: 0.1741 - val_acc: 0.9250
Epoch 2/20
582/582 [==============================] - ETA: 0s - loss: 0.1816 - acc: 0.9262
Epoch 2: val_loss improved from 0.17412 to 0.16627, saving model to my_checkpoint.ckpt
582/582 [==============================] - 120s 207ms/step - loss: 0.1816 - acc: 0.9262 - val_loss: 0.1663 - val_acc: 0.9284
Epoch 3/20
582/582 [==============================] - ETA: 0s - loss: 0.1547 - acc: 0.9369
Epoch 3: val_loss did not improve from 0.16627
582/582 [==============================] - 121s 208ms/step - loss: 0.1547 - acc: 0.9369 - val_loss: 0.1776 - val_acc: 0.9248
Epoch 4/20
582/582 [==============================] - ETA: 0s - loss: 0.1459 - acc: 0.9400
Epoch 4: val_loss did not improve from 0.166

## 학습 완료 후 Load Weights (ModelCheckpoint)

학습이 완료된 후에는 반드시 `load_weights`를 해주어야 합니다.

그렇지 않으면, 열심히 ModelCheckpoint를 만든 의미가 없습니다.

In [13]:
# checkpoint 를 저장한 파일명을 입력합니다.
model.load_weights(checkpoint_path)